# 204: Exampville Destination Choice without Logsums

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [ ]:
# TEST
import warnings

from pytest import approx

import larch as lx

warnings.filterwarnings(action="ignore", category=FutureWarning)

In [ ]:
import numpy as np
import pandas as pd

import larch as lx
from larch import P, X

In this example notebook, we will walk through the estimation of a tour 
destination choice model, without using mode choice logsums.  First, let's load the data files from
our example.

In [ ]:
hh, pp, tour, skims, emp = lx.example(200, ["hh", "pp", "tour", "skims", "emp"])

In [ ]:
hh["INCOME_GRP"] = pd.qcut(hh.INCOME, 3)

## Preprocessing

The alternatives in
the destinations model are much more regular than in the mode choice 
model, as every observation will have a similar set of alternatives
and the utility function for each of those alternatives will share a 
common functional form.  We'll leverage this by using `idca` format 
arrays in our DataTree to make data management simpler.  

The base array we'll start with is the tours, and it only has one 
dimension (cases).  We'll specify the alternatives seperately as `alts`,
and it will be a second dimension, which only has coordinates (alt id's)
and no other data.

In [ ]:
co = lx.Dataset.construct(
    tour.set_index("TOURID"),
    caseid="TOURID",
    alts=skims.TAZ_ID,
)
co

For our destination choice model, we'll also want to use employment data.
This data, as included in our example, has unique 
values only by alternative and not by caseid, so there are only
40 unique rows.
(This kind of structure is common for destination choice models.)

In [ ]:
emp.info()

Then we bundle all our raw data into a `DataTree` structure, 
which is used to collect the right data for estimation.  The
Larch DataTree is a slightly augmented version of the regular
`sharrow.DataTree`.

In [ ]:
tree = lx.DataTree(
    base=co,
    hh=hh.set_index("HHID"),
    person=pp.set_index("PERSONID"),
    emp=emp,
    skims=lx.Dataset.construct.from_omx(skims),
    relationships=(
        "base.TAZ_ID @ emp.TAZ",
        "base.HHID @ hh.HHID",
        "base.PERSONID @ person.PERSONID",
        "hh.HOMETAZ @ skims.otaz",
        "base.TAZ_ID @ skims.dtaz",
    ),
).digitize_relationships()

## Model Definition

Now we can define our choice model, using data from the tree as appropriate.

In [ ]:
m = lx.Model(datatree=tree)
m.title = "Exampville Tour Destination Choice v2"

In [ ]:
m.quantity_ca = (
    +P.EmpRetail_HighInc * X("RETAIL_EMP * (INCOME>50000)")
    + P.EmpNonRetail_HighInc * X("NONRETAIL_EMP") * X("INCOME>50000")
    + P.EmpRetail_LowInc * X("RETAIL_EMP") * X("INCOME<=50000")
    + P.EmpNonRetail_LowInc * X("NONRETAIL_EMP") * X("INCOME<=50000")
)

m.quantity_scale = P.Theta

In [ ]:
m.utility_ca = +P.distance * X.AUTO_DIST

In [ ]:
m.choice_co_code = "base.DTAZ"

In [ ]:
m.plock(EmpRetail_HighInc=0, EmpRetail_LowInc=0)

In [ ]:
# TEST
assert m.availability_any

In [ ]:
mj = m.copy()

## Model Estimation

In [ ]:
m.compute_engine = "numba"

In [ ]:
# TEST
assert m.loglike() == approx(-77777.17321427424)
assert mj.loglike() == approx(-77777.17321427424)

In [ ]:
# TEST
assert m.d_loglike() == approx([-223.95036, -682.1102, 0.0, 0.0, -7406.393, -34762.906])
assert mj.d_loglike() == approx(
    [-223.81805, -681.7803, 0.0, 0.0, -7406.3945, -34767.668], rel=1e-5
)

In [ ]:
result = m.maximize_loglike(stderr=True)

In [ ]:
# TEST
assert result.loglike == approx(-70650.07578452416)
assert result.success
assert result.method == "slsqp"
assert result.n_cases == 20739
assert result.logloss == approx(3.4066288531040145)
import pandas as pd

pd.testing.assert_series_equal(
    result.x.sort_index(),
    pd.Series(
        {
            "EmpNonRetail_HighInc": 1.2453335020460703,
            "EmpNonRetail_LowInc": -1.0893594261458912,
            "EmpRetail_HighInc": 0.0,
            "EmpRetail_LowInc": 0.0,
            "Theta": 0.676440163641688,
            "distance": -0.3347118435209836,
        }
    ).sort_index(),
    rtol=1e-3,
)
assert m.pstderr == approx(
    np.array([0.145749, 0.052355, 0.0, 0.0, 0.009012, 0.003812]),
    rel=1e-3,
)

In [ ]:
resultj = mj.maximize_loglike(stderr=True)

In [ ]:
resultj

In [ ]:
# TEST
assert resultj.loglike == approx(-70650.07578452416)
assert resultj.success
assert resultj.n_cases == 20739
assert resultj.logloss == approx(3.4066288531040145)
import pandas as pd

pd.testing.assert_series_equal(
    pd.Series(resultj.x, index=mj.pnames).sort_index(),
    pd.Series(
        {
            "EmpNonRetail_HighInc": 1.2453335020460703,
            "EmpNonRetail_LowInc": -1.0893594261458912,
            "EmpRetail_HighInc": 0.0,
            "EmpRetail_LowInc": 0.0,
            "Theta": 0.676440163641688,
            "distance": -0.3347118435209836,
        }
    ).sort_index(),
    rtol=1e-2,
)
assert resultj.stderr == approx(
    np.array([0.14442, 0.052348, 0.0, 0.0, 0.009009, 0.003812], dtype=np.float32),
    rel=1e-2,
)

In [ ]:
# TEST
assert mj.bhhh() == approx(
    np.asarray(
        [
            [
                4.89351526e01,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                1.82413773e02,
                2.34888166e02,
            ],
            [
                0.00000000e00,
                3.65410943e02,
                0.00000000e00,
                0.00000000e00,
                -2.06872481e02,
                5.64956295e02,
            ],
            [
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
            ],
            [
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
            ],
            [
                1.82413773e02,
                -2.06872481e02,
                0.00000000e00,
                0.00000000e00,
                1.45100203e04,
                1.20873565e04,
            ],
            [
                2.34888166e02,
                5.64956295e02,
                0.00000000e00,
                0.00000000e00,
                1.20873565e04,
                8.60040357e04,
            ],
        ]
    ),
    rel=0.1,
)

In [ ]:
# TEST
assert m.bhhh() == approx(
    np.asarray(
        [
            [
                4.842784e01,
                0.000000e00,
                0.000000e00,
                0.000000e00,
                1.817289e02,
                2.330331e02,
            ],
            [
                0.000000e00,
                3.653474e02,
                0.000000e00,
                0.000000e00,
                -2.060223e02,
                5.647817e02,
            ],
            [
                0.000000e00,
                0.000000e00,
                0.000000e00,
                0.000000e00,
                0.000000e00,
                0.000000e00,
            ],
            [
                0.000000e00,
                0.000000e00,
                0.000000e00,
                0.000000e00,
                0.000000e00,
                0.000000e00,
            ],
            [
                1.817289e02,
                -2.060223e02,
                0.000000e00,
                0.000000e00,
                1.451344e04,
                1.209207e04,
            ],
            [
                2.330331e02,
                5.647817e02,
                0.000000e00,
                0.000000e00,
                1.209207e04,
                8.600466e04,
            ],
        ]
    ),
    rel=0.01,
    abs=0.00001,
)

## Model Visualization

For destination choice and similar type models, it might be beneficial to
review the observed and modeled choices, and the relative distribution of
these choices across different factors.  For example, we would probably want
to see the distribution of travel distance.  The `Model` object includes
a built-in method to create this kind of visualization.

In [ ]:
m.histogram_on_idca_variable("AUTO_DIST")

In [ ]:
m.histogram_on_idca_variable("RETAIL_EMP")

The `histogram_on_idca_variable` has a variety of options,
for example to control the number and range of the histogram bins:

In [ ]:
m.histogram_on_idca_variable("AUTO_DIST", bins=40, span=(0, 10))

Subsets of the observations can be pulled out, to observe the 
distribution conditional on other `idco` factors, like income.

In [ ]:
m.histogram_on_idca_variable(
    "AUTO_DIST",
    x_label="Distance (miles)",
    bins=26,
    span=(0, 13),
    filter_co="INCOME<10000",
)

## Semi-Aggregate Data

Some choice models are based on only a limited number of disaggregate data dimensions.
The destination choice model shown above is an example of this, where the only explanatory
data used is the home zone id (which determines the distances to all the various destination
alternatives) and whether the household income is low or not.  Our original source data has
more than 20,000 case observations, but there can only be up to 80 actual unique choice 
situations (40 zones, times 2 income categories).  Given this simple model, it can be
much more efficient to aggregate the data along all the relevant dimensions.

In [ ]:
tour_plus = tour.join(hh.set_index("HHID")[["HOMETAZ", "INCOME"]], on="HHID")
tour_plus["LOW_INCOME"] = tour_plus.INCOME < 50_000
tour_agg = (
    tour_plus.groupby(["HOMETAZ", "DTAZ", "LOW_INCOME"])
    .size()
    .unstack("DTAZ")
    .fillna(0)
)

In [ ]:
# j = tour_agg.reset_index(drop=True)
# lx.DataArray(j.values, dims=("index", "DTAZ"), coords={"index": j.index, "DTAZ": j.columns})

In [ ]:
agg_dataset = lx.Dataset.construct.from_idco(
    tour_agg.index.to_frame().reset_index(drop=True)
)
j = tour_agg.reset_index(drop=True)
agg_dataset = agg_dataset.assign(
    destinations=lx.DataArray(
        j.values,
        dims=("index", "DTAZ"),
        coords={"index": j.index, "DTAZ": j.columns},
    )
)
agg_dataset.dc.ALTID = "DTAZ"
agg_dataset

In [ ]:
agg_tree = lx.DataTree(
    base=agg_dataset,
    emp=emp,
    skims=lx.Dataset.construct.from_omx(skims),
    relationships=(
        "base.DTAZ @ emp.TAZ",
        "base.HOMETAZ @ skims.otaz",
        "base.DTAZ @ skims.dtaz",
    ),
)

In [ ]:
mg = lx.Model(datatree=agg_tree, compute_engine="numba")
mg.title = "Exampville Semi-Aggregate Destination Choice"

In [ ]:
mg.quantity_ca = (
    +P.EmpRetail_HighInc * X("RETAIL_EMP * (1-LOW_INCOME)")
    + P.EmpNonRetail_HighInc * X("NONRETAIL_EMP") * X("(1-LOW_INCOME)")
    + P.EmpRetail_LowInc * X("RETAIL_EMP") * X("LOW_INCOME")
    + P.EmpNonRetail_LowInc * X("NONRETAIL_EMP") * X("LOW_INCOME")
)

mg.quantity_scale = P.Theta

In [ ]:
mg.utility_ca = +P.distance * X.AUTO_DIST

In [ ]:
mg.choice_ca_var = "base.destinations"

In [ ]:
mg.plock(EmpRetail_HighInc=0, EmpRetail_LowInc=0)

In [ ]:
# TEST
assert mg.loglike() == approx(-77777.17321427427)

In [ ]:
# TEST
assert mg.d_loglike() == approx([-223.95016, -682.1102, 0, 0, -7406.389, -34762.91])

In [ ]:
result = mg.maximize_loglike(stderr=True)
result

In [ ]:
# TEST
assert result.loglike == approx(-70650.07578452416)
assert result.success
assert result.method == "slsqp"
assert result.n_cases == 79
assert result.logloss == approx(3.4066288531040145)
import pandas as pd

pd.testing.assert_series_equal(
    result.x.sort_index(),
    pd.Series(
        {
            "EmpNonRetail_HighInc": 1.2453335020460703,
            "EmpNonRetail_LowInc": -1.0893594261458912,
            "EmpRetail_HighInc": 0.0,
            "EmpRetail_LowInc": 0.0,
            "Theta": 0.676440163641688,
            "distance": -0.3347118435209836,
        }
    ).sort_index(),
    rtol=1e-2,
)
assert m.pstderr == approx(
    np.array([0.145749, 0.052355, 0.0, 0.0, 0.009012, 0.003812]),
    rel=1e-3,
)

In [ ]:
# TEST
assert mg.total_weight() == approx(20739.0)
assert mg.n_cases == 79

In [ ]:
# TEST
assert mg.bhhh() == approx(
    np.array(
        [
            [
                4.92276067e01,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                1.82962749e02,
                2.35657294e02,
            ],
            [
                0.00000000e00,
                3.66140769e02,
                0.00000000e00,
                0.00000000e00,
                -2.02327809e02,
                5.64802080e02,
            ],
            [
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
            ],
            [
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
                0.00000000e00,
            ],
            [
                1.82962749e02,
                -2.02327809e02,
                0.00000000e00,
                0.00000000e00,
                1.45027493e04,
                1.20949516e04,
            ],
            [
                2.35657294e02,
                5.64802080e02,
                0.00000000e00,
                0.00000000e00,
                1.20949516e04,
                8.59976140e04,
            ],
        ]
    ),
    rel=1e-2,
)